# Jeu de devinette : Père Fouras vs Laurent Jalabert

Dans ce notebook, nous allons simuler le duel légendaire entre le Père Fouras et Laurent Jalabert en utilisant Semantic Kernel avec des agents conversationnels.

In [ ]:
# Bloc 1 - Installation et imports
%pip install semantic-kernel python-dotenv --quiet
import os
import logging
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import KernelArguments

# Configuration des logs
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('FortBoyard')

# Chargement des variables d'environnement
load_dotenv()


## 1. Installation et configuration

Cette cellule prepare l'environnement :
- **semantic-kernel** : SDK pour orchestrer les agents LLM
- **python-dotenv** : Chargement securise des cles API depuis `.env`
- **Logging** : Configuration des logs pour suivre la conversation

Le fichier `.env` doit contenir `OPENAI_API_KEY` pour l'authentification.

## Configuration des agents

### Le mot a deviner

La variable `MOT_A_DEVINER` contient le mot que Laurent Jalabert doit trouver. Ce mot est injecte dans le prompt du Pere Fouras mais reste cache pour l'agent devineur.

La fonction `create_kernel()` cree une instance du Kernel avec le service OpenAI configure.

In [ ]:
# Bloc 2 - Création du kernel
MOT_A_DEVINER = "anticonstitutionnellement"

def create_kernel():
    kernel = Kernel()
    kernel.add_service(OpenAIChatCompletion(
        service_id="openai",
        ai_model_id="gpt-4o-mini",
        api_key=os.getenv("OPENAI_API_KEY")
    ))
    return kernel


### Conception des prompts

Les prompts systeme definissent la personnalite de chaque agent :
- **Pere Fouras** : Donne des indices enigmatiques, parle en charades, ne revele jamais le mot
- **Laurent Jalabert** : Pose des questions fermees (oui/non) pour deviner

Le mot a deviner est injecte dynamiquement dans le prompt du Pere Fouras via une f-string.

## Définition des prompts

### Instanciation des agents

Chaque agent possede :
- Son propre **Kernel** (instance independante)
- Un **nom** unique pour l'identification dans les logs
- Des **instructions** definissant son comportement

Les deux agents utiliseront le meme modele OpenAI mais avec des roles differents.

In [ ]:
# Bloc 3 - Prompts des agents
PERE_FOURAS_PROMPT = f"""
Tu es le Père Fouras de Fort Boyard. 
Tu dois faire deviner le mot '{MOT_A_DEVINER}'. 
Utilise des charades et réponses énigmatiques. 
Ne révèle jamais directement le mot !
"""

LAURENT_JALABERT_PROMPT = """
Tu es Laurent Jalabert. 
Tu dois deviner le mot en posant des questions fermées (Oui/Non).
Sois perspicace et stratégique dans tes questions.
"""

### Strategie de terminaison personnalisee

La classe `FortBoyardTerminationStrategy` herite de `TerminationStrategy` et definit quand le jeu se termine :
- **Condition** : Le mot a deviner apparait dans le dernier message
- **Methode** : `should_terminate()` est appelee apres chaque tour

Cela illustre comment personnaliser le comportement d'arret d'un `AgentGroupChat`.

## Création des agents avec stratégies personnalisées

### Configuration du groupe de discussion

Le `AgentGroupChat` orchestre les deux agents :
- **agents** : Liste des participants (Pere Fouras et Laurent Jalabert)
- **termination_strategy** : Notre strategie personnalisee
- **maximum_iterations** : Limite de securite (20 tours max)

Les agents parlent en alternance jusqu'a ce que le mot soit devine ou la limite atteinte.

In [ ]:
# Bloc 4 - Définition des agents
pere_fouras = ChatCompletionAgent(
    kernel=create_kernel(),
    # service_id="openai",
    name="Pere_Fouras",
    instructions=PERE_FOURAS_PROMPT,
)

laurent_jalabert = ChatCompletionAgent(
    kernel=create_kernel(),
    # service_id="openai",
    name="Laurent_Jalabert",
    instructions=LAURENT_JALABERT_PROMPT,
)

### Execution du jeu

La fonction `jouer_partie()` lance la conversation :
1. Affiche le mot a deviner (pour le debug)
2. Itere sur `chat.invoke()` qui retourne chaque message en streaming
3. Log chaque intervention avec le role de l'agent
4. S'arrete quand la strategie de terminaison retourne `True`

Executez cette cellule pour voir le duel en action !

## Stratégie de terminaison personnalisée

In [ ]:
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent

# Bloc 5 - Logique de terminaison
class FortBoyardTerminationStrategy(TerminationStrategy):
    """Arrête la partie si le mot est deviné"""
    
    async def should_terminate(
        self, 
        agent: ChatCompletionAgent, 
        history: list[ChatMessageContent], 
        cancellation_token = None
    ) -> bool:
        if not history:
            return False
        
        last_message = str(history[-1].content).lower()
        return MOT_A_DEVINER in last_message

## Configuration du groupe de discussion

In [ ]:
# Bloc 6 - Configuration corrigée
chat = AgentGroupChat(
    agents=[pere_fouras, laurent_jalabert],
    termination_strategy=FortBoyardTerminationStrategy(
        agents=[laurent_jalabert],  # Définit explicitement les agents
        maximum_iterations=20       # Définit le nombre max d'itérations
    )
)


## Lancement de la partie !

In [ ]:
from semantic_kernel.contents import AuthorRole, ChatMessageContent

# Bloc 7 - Exécution du jeu
async def jouer_partie():
    logger.info("🚀 Départ du duel Père Fouras vs Laurent Jalabert !")
    logger.info(f"Mot à deviner : {MOT_A_DEVINER.upper()}")
    
    # Ajout du message système directement dans l'historique du chat
    # await chat.add_chat_message(ChatMessageContent(role=AuthorRole.DEVELOPER, content="Nouvelle partie commencée !"))

    
    async for message in chat.invoke():
        role = message.role
        # name = message.author_name or "System"
        logger.info(f"[{role}] : {message.content}")
    
    logger.info("🎉 Partie terminée !")

await jouer_partie()


## Conclusion du jeu Fort Boyard

Ce notebook a illustre plusieurs concepts avances de Semantic Kernel :

### Concepts techniques demontres

1. **Agents conversationnels specialises** : Chaque agent possede sa propre personnalite et son role
2. **Strategie de terminaison personnalisee** : Detection automatique du succes (mot devine)
3. **AgentGroupChat** : Orchestration d'un dialogue structure entre deux agents
4. **Gestion du contexte** : Maintien de l'historique de conversation pour coherence

### Applications pratiques

Ce pattern peut etre adapte pour :
- **Jeux educatifs** : Quiz interactifs, devinettes pedagogiques
- **Interviews automatisees** : Un agent questionne, l'autre repond
- **Debat structure** : Deux agents defensent des positions opposees
- **Scenarios de formation** : Simulation de conversations professionnelles

### Points d'amelioration possibles

- Ajouter un compteur de tentatives avec limite
- Implementer un systeme de score base sur le nombre de questions
- Enrichir les indices du Pere Fouras avec des images ou sons
- Logger la conversation complete pour analyse posterieure